# Emotion Classifier with Pytorch


In [5]:
import torch
from RAVDESS_data import RavdessDataset

In [7]:
ANNOTATIONS_FILE = "/Users/stephen/Desktop/Emotion_Speech_Detection/data/metadata.csv"
AUDIO_DIR = "/Users/stephen/Desktop/Emotion_Speech_Detection/data/RAVDESS"
SAMPLE_RATE = 16000
NUM_SAMPLES = 16000*4

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")


rav = RavdessDataset(
    annotations_file = ANNOTATIONS_FILE, 
    audio_dir = AUDIO_DIR, 
    target_sample_rate = SAMPLE_RATE, 
    num_samples = NUM_SAMPLES,
    device = device
)

Using device cpu


In [10]:
rav[1][0]

IndexError: single positional indexer is out-of-bounds